In [1]:
from sklearn.datasets import fetch_openml
import numpy as np
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
mnist.target = mnist.target.astype(np.uint8)
X=mnist["data"]
y=mnist["target"]

In [6]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pickle
# def plot_clusters(X, y=None):
#     plt.scatter(X[:, 0], X[:, 1], c=y , s=1)
#     plt.xlabel("$x_1$", fontsize=14)
#     plt.ylabel("$x_2$", fontsize=14, rotation=0)
# 
# plt.figure(figsize=(8, 4))
# plot_clusters(X,y) # zobaczmy skupiska
# f = 'blobs_plot.png'
# plt.savefig(f)
# print(f)
silhouette_scores = []
for i in [8, 9, 10, 11, 12]:
    kmeans = KMeans(n_clusters=i, n_init=10, random_state=42)
    kmeans.fit(X)
    labels = kmeans.labels_
    silhouette_ = silhouette_score(X, labels)
    silhouette_scores.append(silhouette_)

with open('kmeans_sil.pkl', 'wb') as f:
    pickle.dump(silhouette_scores, f)
    

In [11]:
from sklearn.metrics import confusion_matrix
kmeans = KMeans(n_clusters=10, n_init=10, random_state=42)
kmeans.fit(X)
conf_mx = confusion_matrix(y, kmeans.predict(X))
print(conf_mx)
list_of_indexes = []
for row in conf_mx:
    idx = np.argmax(row)
    list_of_indexes.append(idx)
print(sorted(set(list_of_indexes)))
with open('kmeans_argmax.pkl', 'wb') as f:
    pickle.dump(sorted(set(list_of_indexes)), f)

[[ 290    2    7  162 1265   72    9    4   39 5053]
 [   8 4293   11    7    7    8   10 3526    7    0]
 [ 323  423   78  147  246  201 4863  436  216   57]
 [4581  449   45   31  460 1083  217   58  193   24]
 [   0  178 2173  168  288   17   29  234 3728    9]
 [2129  155  215   67 1812 1156    7  280  432   60]
 [  38  190    4 4324 2070   14   53   45   67   71]
 [   6  372 4399    4   12   18   53  314 2094   21]
 [1212  335  193   51  292 4115   53  330  208   36]
 [  87  261 2849   16   31   87   19   95 3462   51]]
[0, 1, 2, 3, 5, 6, 8, 9]


In [14]:
from sklearn.cluster import DBSCAN

distances = []
for i in range(300):
    for j in range(len(X)):
        dst = np.linalg.norm(X[i] - X[j])
        if dst != 0:
            distances.append(dst)

distances = sorted(distances)


[304.37641170103836, 317.5893575043093, 328.7658741414626, 333.4546445920344, 352.89800226127664, 355.1774204534967, 358.07401469528617, 359.64287842247063, 360.42474942767177]


In [15]:
with open('dist.pkl', 'wb') as file:
    pickle.dump(distances[:10], file)

In [16]:
print(distances[:10])

[279.26152617215286, 304.37641170103836, 317.5893575043093, 328.7658741414626, 333.4546445920344, 352.89800226127664, 355.1774204534967, 358.07401469528617, 359.64287842247063, 360.42474942767177]


In [18]:
avg_distance = sum(distances[:3])/3
print(avg_distance)
s = avg_distance
eps_vals = []
while s <= avg_distance+(avg_distance*0.1):
    eps_vals.append(s)
    s = s + (s*0.04)

print(eps_vals)

300.40909845916684
[300.40909845916684, 312.4254623975335, 324.92248089343485]


In [20]:
unique_labels = []
for e in eps_vals:
    dbscan = DBSCAN(eps=e, min_samples=5)
    dbscan.fit(X)
    unique_labels.append(len(set(dbscan.labels_)))

print(unique_labels)

[4, 7, 23]


In [21]:
with open('dbscan_len.pkl', 'wb') as file:
    pickle.dump(unique_labels, file)